In [3]:
import pandas as pd
import numpy as np

# 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [21]:
attendance_df = pd.read_csv('untidy-data/attendance.csv')
attendance_df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [169]:
attendance_melt = attendance_df.melt(id_vars=['Unnamed: 0'], var_name='date', value_name='attend')
attendance_melt.head()

,Unnamed: 0,date,attend
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [170]:
mapping = {'A': 1, 'H': .5, 'T': .1, 'P': 0}

# Replace values in the 'column_name' column using the mapping
attendance_melt['attend'] = attendance_melt['attend'].map(mapping)

In [171]:
attendance_melt.head()

,Unnamed: 0,date,attend
0,Sally,2018-01-01,0.0
1,Jane,2018-01-01,1.0
2,Billy,2018-01-01,1.0
3,John,2018-01-01,0.0
4,Sally,2018-01-02,0.1


In [172]:
1-attendance_melt.groupby('Unnamed: 0').mean()

/var/folders/qh/kjmdb2zx3n3b2ng_n4q3cyn00000gn/T/ipykernel_45846/1960734550.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  1-attendance_melt.groupby('Unnamed: 0').mean()


,attend
Unnamed: 0,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


You should end up with something like this:

    `name
    Billy    0.5250
    Jane     0.6875
    John     0.9125
    Sally    0.7625
    Name: grade, dtype: float64`


<hr style="border: 2px solid #000000;">

# 2. Coffee Levels

* a. Read the coffee_levels table.

In [27]:
cof_df = pd.read_csv('untidy-data/coffee_levels.csv')
cof_df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


* b. Transform the data so that each carafe is in it's own column.

In [29]:
cof_df.pivot_table(values='coffee_amount', index='hour', columns='coffee_carafe')

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


* c. Is this the best shape for the data?

In [ ]:
No

<hr style="border: 2px solid #000000;">

# 3. Cake Recipes

* a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [31]:
cake_df = pd.read_csv('untidy-data/cake_recipes.csv')
cake_df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


* b. Tidy the data as necessary.

In [32]:
cake_melt = cake_df.melt(id_vars=['recipe:position'])
cake_melt.head()

,recipe:position,variable,value
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [36]:
#
cake_melt[['recipe', 'layer']] = cake_melt['recipe:position'].str.split(':', expand=True)
cake_melt.head()

,recipe:position,variable,value,recipe,layer
0,a:bottom,225,61.738655,a,bottom
1,a:top,225,51.709751,a,top
2,b:bottom,225,57.095320,b,bottom
3,b:top,225,82.455004,b,top
4,c:bottom,225,96.470207,c,bottom


In [108]:
#
cake_tidy = cake_melt.pivot_table(index=['layer', 'variable'],
                      columns=['recipe'],
                      values=['value']).reset_index()
cake_tidy

layer variable      value                                 
recipe                           a          b          c          d
0       bottom      225  61.738655  57.095320  96.470207  52.799753
1       bottom      250  53.912627  61.904369  52.001358  58.670419
2       bottom      275  74.414730  61.196980  92.893227  51.747686
3       bottom      300  98.786784  99.248541  65.473084  56.183110
4          top      225  51.709751  82.455004  71.306308  96.873178
5          top      250  52.009735  95.224151  82.795477  76.101363
6          top      275  68.576858  98.594881  92.098049  59.571620
7          top      300  50.228470  58.169349  53.960273  50.971626

* c. Which recipe, on average, is the best?

In [110]:
for col in cake_tidy.iloc[:, 2:6]:
    print(f'Recipe {col} = {cake_tidy[col].mean()}')

Recipe ('value', 'a') = 63.922201303273
Recipe ('value', 'b') = 76.73607424644436
Recipe ('value', 'c') = 75.87474783650127
Recipe ('value', 'd') = 62.86484431129857


In [55]:
cake_tidy.iloc[:, 2:5] # return the columns i need to do my loop

value                      
recipe          a          b          c
0       61.738655  57.095320  96.470207
1       53.912627  61.904369  52.001358
2       74.414730  61.196980  92.893227
3       98.786784  99.248541  65.473084
4       51.709751  82.455004  71.306308
5       52.009735  95.224151  82.795477
6       68.576858  98.594881  92.098049
7       50.228470  58.169349  53.960273

* d. Which oven temperature, on average, produces the best results?

In [63]:
cake_tidy.groupby('variable').max().mean()

/var/folders/qh/kjmdb2zx3n3b2ng_n4q3cyn00000gn/T/ipykernel_45846/1505083730.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  cake_tidy.groupby('variable').max().mean()
/var/folders/qh/kjmdb2zx3n3b2ng_n4q3cyn00000gn/T/ipykernel_45846/1505083730.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cake_tidy.groupby('variable').max().mean()


       recipe
value  a         72.213199
       b         93.880644
       c         84.407999
       d         72.182318
dtype: float64

* e. Which combination of recipe, rack position, and temperature gives the best result?

In [66]:
cake_tidy.max()

          recipe
layer                     top
variable                  300
value     a         98.786784
          b         99.248541
          c         96.470207
          d         96.873178
dtype: object

In [81]:
cake_tidy.max()[3]

99.2485405378462

In [85]:
cake_tidy[(cake_tidy == cake_tidy.max()[3]).any(axis=1)]

layer variable      value                                
recipe                           a          b          c         d
3       bottom      300  98.786784  99.248541  65.473084  56.18311

<hr style="border: 2px solid #000000;">

## Bonus: explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.

In [100]:
score_df = pd.read_excel('untidy-data/dem_score.xlsx')
score_df.head()

,country,1952,1957,1962,1967,1972,1977,1982,1987,1992
0,Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
1,Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
2,Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
3,Australia,10,10,10,10,10,10,10,10,10
4,Austria,10,10,10,10,10,10,10,10,10


In [101]:
score_df = score_df.melt(id_vars=['country'], var_name='year', value_name='value')
score_df.head()

,country,year,value
0,Albania,1952,-9
1,Argentina,1952,-9
2,Armenia,1952,-9
3,Australia,1952,10
4,Austria,1952,10


<hr style="border: 2px solid #000000;">

In [102]:
gap_df = pd.read_csv('untidy-data/gapminder1.csv')
gap_df.head()

,year,country,measure,measurement
0,1955,Afghanistan,pop,8891209.0
1,1960,Afghanistan,pop,9829450.0
2,1965,Afghanistan,pop,10997885.0
3,1970,Afghanistan,pop,12430623.0
4,1975,Afghanistan,pop,14132019.0


In [126]:
gap_df = gap_df.pivot_table(index=['measure', 'year'],
                      columns=['country'],
                      values=['measurement']).reset_index()
gap_df.head()

measure  year measurement                                     \
country                  Afghanistan Argentina  Aruba Australia Austria   
0        fertility  1955         7.7    3.1265  5.150     3.406    2.52   
1        fertility  1960         7.7    3.0895  4.399     3.274    2.78   
2        fertility  1965         7.7    3.0490  3.301     2.871    2.53   
3        fertility  1970         7.7    3.1455  2.651     2.535    2.02   
4        fertility  1975         7.7    3.4400  2.450     1.989    1.64   

                                     ...                                   \
country Bahamas Bangladesh Barbados  ... Rwanda Saudi Arabia South Africa   
0         4.305       6.76     4.67  ...  8.000        7.175         6.50   
1         4.503       6.85     4.26  ...  8.100        7.257         6.30   
2         3.794       6.60     3.45  ...  8.200        7.257         5.70   
3         3.444       6.15     2.74  ...  8.290        7.298         5.47   
4         3.221       5.60     2.19  ...  8.492        7.278         5.00   

                                                                           \
country South Korea Spain Switzerland Turkey United Kingdom United States   
0              3.80  2.75        2.34  6.600           2.49         3.706   
1              3.41  2.89        2.51  6.190           2.81         3.314   
2              4.09  2.92        2.27  5.700           2.52         2.545   
3              3.72  2.86        1.82  5.300           2.04         2.016   
4              2.58  2.57        1.53  4.715           1.72         1.788   

                   
country Venezuela  
0          6.4585  
1          6.6570  
2          5.9045  
3          4.9410  
4          4.4685  

[5 rows x 65 columns]

<hr style="border: 2px solid #000000;">

In [160]:
gap2_df = pd.read_csv('untidy-data/gapminder2.csv')
gap2_df.head()

,country,life_expect_1955,life_expect_1960,life_expect_1965,life_expect_1970,life_expect_1975,life_expect_1980,life_expect_1985,life_expect_1990,life_expect_1995,...,pop_1960,pop_1965,pop_1970,pop_1975,pop_1980,pop_1985,pop_1990,pop_1995,pop_2000,pop_2005
0,Afghanistan,30.332,31.997,34.020,36.088,38.438,39.854,40.822,41.674,41.763,...,9829450,10997885,12430623,14132019,15112149,13796928,14669339,20881480,23898198,29928987
1,Argentina,64.399,65.142,65.634,67.065,68.481,69.942,70.774,71.868,73.275,...,20616009,22283100,23962313,26081880,28369799,30675059,33022202,35311049,37497728,39537943
2,Aruba,64.381,66.606,68.336,70.941,71.830,74.116,74.494,74.108,73.011,...,57203,59020,59039,59390,60266,64129,66653,67836,69539,71566
3,Australia,70.330,70.930,71.100,71.930,73.490,74.740,76.320,77.560,78.830,...,10361273,11439384,12660160,13771400,14615900,15788300,17022133,18116171,19164620,20090437
4,Austria,67.480,69.540,70.140,70.630,72.170,73.180,74.940,76.040,77.510,...,7047437,7270889,7467086,7578903,7549433,7559776,7722953,8047433,8113413,8184691


In [161]:
gap2_melt = gap2_df.melt(id_vars=['country'])
gap2_melt.head()



,country,variable,value
0,Afghanistan,life_expect_1955,30.332
1,Argentina,life_expect_1955,64.399
2,Aruba,life_expect_1955,64.381
3,Australia,life_expect_1955,70.330
4,Austria,life_expect_1955,67.480


In [162]:
# gap2_melt[['measure', 'year']] = 
gap2_melt['variable'].str.split([-5], expand=True)
# cake_melt.head()

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1381,NaN
1382,NaN
1383,NaN
1384,NaN


In [163]:
gap2_melt['variable'].str.split('_')


0       [life, expect, 1955]
1       [life, expect, 1955]
2       [life, expect, 1955]
3       [life, expect, 1955]
4       [life, expect, 1955]
                ...         
1381             [pop, 2005]
1382             [pop, 2005]
1383             [pop, 2005]
1384             [pop, 2005]
1385             [pop, 2005]
Name: variable, Length: 1386, dtype: object

In [164]:
# Split the string in the 'variable' column at a specific position
split_cols = gap2_melt['variable'].str.split([-5], expand=True)

# Assign the split columns to new columns in the DataFrame
gap2_melt['column1'] = split_cols[0]
gap2_melt['column2'] = split_cols[1]

# Remove the original 'variable' column if desired
gap2_melt.drop('variable', axis=1, inplace=True)

# Print the updated DataFrame
print(gap2_melt)

KeyError: 1

In [165]:
'string'[-2]

'n'

In [166]:
gap2_melt['variable'].str.slice(stop=-5)

0       life_expect
1       life_expect
2       life_expect
3       life_expect
4       life_expect
           ...     
1381            pop
1382            pop
1383            pop
1384            pop
1385            pop
Name: variable, Length: 1386, dtype: object

In [158]:
split_cols = gap2_melt['variable'].str.slice(stop=-5)

# Assign the split columns to new columns in the DataFrame
gap2_melt['column1'] = split_cols

# Print the updated DataFrame
print(gap2_melt)

             country          variable         value      column1
0        Afghanistan  life_expect_1955  3.033200e+01  life_expect
1          Argentina  life_expect_1955  6.439900e+01  life_expect
2              Aruba  life_expect_1955  6.438100e+01  life_expect
3          Australia  life_expect_1955  7.033000e+01  life_expect
4            Austria  life_expect_1955  6.748000e+01  life_expect
...              ...               ...           ...          ...
1381     Switzerland          pop_2005  7.489370e+06          pop
1382          Turkey          pop_2005  6.966056e+07          pop
1383  United Kingdom          pop_2005  6.044146e+07          pop
1384   United States          pop_2005  2.957341e+08          pop
1385       Venezuela          pop_2005  2.537528e+07          pop

[1386 rows x 4 columns]
